In [1]:
import pandas as pd
from NlpStudyFunctions import CleanDataframe
from sklearn.model_selection import train_test_split

train_df = pd.read_csv(r'../data/raw/train.csv')

#clean the df
#train_df = CleanDataframe(train_df_raw,'text')

#split train-test set
x_train, x_test, y_train, y_test = train_test_split(train_df['text'].values, train_df['target'].values, test_size=0.2, random_state=42)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\guibe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\guibe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(x_train)

#x_train_list = x_train.tolist() if isinstance(x_train, np.ndarray) else x_train
#x_test_list = x_test.tolist() if isinstance(x_test, np.ndarray) else x_test


x_train_sequences = tokenizer.texts_to_sequences(x_train)
x_test_sequences = tokenizer.texts_to_sequences(x_test)

# Calculate the maximum sequence length
max_sequence_length = max(len(x) for x in x_train_sequences)

# Pad sequences to the same length
x_train_padded = pad_sequences(x_train_sequences, maxlen=max_sequence_length)
x_test_padded = pad_sequences(x_test_sequences, maxlen=max_sequence_length)

In [4]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Bidirectional, Conv1D, MaxPooling1D, Dense, Dropout
from keras.optimizers import Adam

# Assuming num_words and max_sequence_length are defined
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=200, input_length=max_sequence_length))
model.add(LSTM(128, return_sequences=True, dropout=0.2))
model.add(LSTM(512, dropout=0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train_padded, y_train, validation_data=(x_test_padded, y_test), epochs=10, batch_size=32)

c:\Users\guibe\anaconda3\envs\kaggle_project\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 27s 110ms/step - accuracy: 0.6519 - loss: 0.6218 - val_accuracy: 0.7951 - val_loss: 0.4825
Epoch 2/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 22s 117ms/step - accuracy: 0.8752 - loss: 0.3359 - val_accuracy: 0.7905 - val_loss: 0.5049
Epoch 3/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 24s 127ms/step - accuracy: 0.9214 - loss: 0.2077 - val_accuracy: 0.7663 - val_loss: 0.6366
Epoch 4/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 23s 121ms/step - accuracy: 0.9542 - loss: 0.1264 - val_accuracy: 0.7610 - val_loss: 0.7163
Epoch 5/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 22s 117ms/step - accuracy: 0.9659 - loss: 0.0937 - val_accuracy: 0.7656 - val_loss: 0.8269
Epoch 6/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 22s 114ms/step - accuracy: 0.9758 - loss: 0.0673 - val_accuracy: 0.7768 - val_loss: 1.0924
Epoch 7/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 22s 114ms/step - accuracy: 0.9793 - loss: 0.0512 - val_accuracy: 0.7492 - val_loss: 1.1410
Epoch 8/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 22s 115ms/step - accuracy: 0.9831 - loss: 0

In [5]:
loss, accuracy = model.evaluate(x_test_padded, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.7400 - loss: 2.0069
Test Loss: 1.8513
Test Accuracy: 0.7557


In [6]:
#save model
#model.save(r'../models/DL_MODEL.keras')

In [16]:
import keras

text_test_df = pd.read_csv(r'../data/raw/test.csv')

text_test_df = text_test_df['text'].values

test_df_sequences = tokenizer.texts_to_sequences(text_test_df)

# Pad sequences to the same length
test_df_padded = pad_sequences(test_df_sequences, maxlen=max_sequence_length)

#model = keras.models.load_model(r'../models/DL_MODEL.keras')

pred = model.predict(test_df_padded)

102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step


In [27]:
import numpy as np
pred2=[np.argmax(x) for x in pred]

test = pd.read_csv(r'../data/raw/test.csv')
test["target"]=pred2
test.head()
submission = test[['id','target']]

In [28]:
submission.to_csv(r'../data/final/submission.csv',index=False)

submission = pd.read_csv(r'../data/final/submission.csv')
submission.describe()

,id,target
count,3263.000000,3263.000000
mean,5427.152927,0.539074
std,3146.427221,0.498547
min,0.000000,0.000000
25%,2683.000000,0.000000
50%,5500.000000,1.000000
75%,8176.000000,1.000000
max,10875.000000,1.000000


In [29]:
submission["target"].value_counts()

target
1    1759
0    1504
Name: count, dtype: int64

In [30]:
submission.head()

,id,target
0,0,1
1,2,0
2,3,0
3,9,0
4,11,1


In [31]:
import kaggle

# Replace 'submission.csv' with the path to your submission file
submission_file = r'../data/final/submission.csv'

# Replace 'Message' with your submission message
submission_message = 'DL model try'

# Call the submit function from kaggle package
kaggle.api.competition_submit(submission_file, submission_message, competition='nlp-getting-started')

100%|██████████| 25.4k/25.4k [00:00<00:00, 50.7kB/s]


Successfully submitted to Natural Language Processing with Disaster Tweets